In [1]:
import numpy as np 
import sympy

# 每个单元的形函数N、几何矩阵B、应力矩阵S、刚度矩阵k的获取函数

In [2]:
def Shape_function_matrix(xi,yi,xj,yj,xm,ym):
    """    
    输入单元节点 i j m 三点的坐标
    输出：单元形函数N
    """
    x=sympy.Symbol("x",real=True)
    y=sympy.Symbol("y",real=True)

    ai=xj*ym-xm*yi
    bi=yj-ym
    ci=-xj+xm

    aj=xm*yi-xi*ym
    bj=ym-yi
    cj=-xm+xi

    am=xi*yj-xj*yi
    bm=yi-yj
    cm=-xi+xj

    A2=np.linalg.det(np.array([ [1,xi,yi],[1,xj,yj],[1,xm,ym] ]))

    Ni=(ai+bi*x+ci*y)/A2
    Nj=(aj+bj*x+cj*y)/A2
    Nm=(am+bm*x+cm*y)/A2
  
    N = np.array([ [Ni,0,Nj,0,Nm,0],[0,Ni,0,Nj,0,Nm] ])    
    return N

#test Shape_function_matrix

Shape_function_matrix(1,0,0,1,0,0)

In [3]:
def Geometric_matrix(xi,yi,xj,yj,xm,ym):
    """
    输入单元节点 i j m 三点的坐标
    输出：单元几何矩阵B
    
    """
    #ai=xj*ym-xm*yi
    bi=yj-ym
    ci=-xj+xm

    #aj=xm*yi-xi*ym
    bj=ym-yi
    cj=-xm+xi

    #am=xi*yj-xj*yi
    bm=yi-yj
    cm=-xi+xj

    B=np.array([ [bi,0,bj,0,bm,0],[0,ci,0,cj,0,cm],[ci,bi,cj,bj,cm,bm] ])
    
    return B
#Geometric_matrix(1,0,0,1,0,0)

In [4]:
def Stress_matrix(xi,yi,xj,yj,xm,ym,E=2*10**11,u=0.3):
    """
    输入单元节点 i j m 三点的坐标  E u
    输出：单元应力矩阵S
    
    """
    #ai=xj*ym-xm*yi
    bi=yj-ym
    ci=-xj+xm

    #aj=xm*yi-xi*ym
    bj=ym-yi
    cj=-xm+xi

    #am=xi*yj-xj*yi
    bm=yi-yj
    cm=-xi+xj
    A2=np.linalg.det(np.array([ [1,xi,yi],[1,xj,yj],[1,xm,ym] ]))  #单元面积的二倍

    # # B=np.array([ [bi,0,bj,0,bm,0],[0,ci,0,cj,0,cm],[ci,bi,cj,bj,cm,bm] ])
    # # D=( E/(1-u**2) )*np.array([ [1,u,0],[u,1,0],[0,0,(1-u)/2] ]) #弹性矩阵
    # # S=np.dot(D,B)

    S=np.array([ [bi,u*ci,bj,u*cj,bm,u*cm],[u*bi,ci,u*bj,cj,u*bm,cm],
                [(1-u)*0.5*ci,(1-u)*0.5*bi,(1-u)*0.5*cj,(1-u)*0.5*bj,(1-u)*0.5*cm,(1-u)*0.5*bm] ])*E/ ( A2*(1-u**2) )
    return S
# Stress_matrix(1,0,0,1,0,0)

In [5]:
def Element_stiffness_matrix(xi,yi,xj,yj,xm,ym,E=2*10**11,u=0.3,h=1):
    """
    输入单元节点 i j m 三点的坐标  杨氏模量E 泊松比u  单元厚度：h
    输出：单元刚度矩阵k

    """
    #ai=xj*ym-xm*yi
    bi=yj-ym
    ci=-xj+xm

    #aj=xm*yi-xi*ym
    bj=ym-yi
    cj=-xm+xi

    #am=xi*yj-xj*yi
    bm=yi-yj
    cm=-xi+xj
    A2=np.linalg.det(np.array([ [1,xi,yi],[1,xj,yj],[1,xm,ym] ])) #单元面积的二倍
    A=A2/2

    B=np.array([ [bi,0,bj,0,bm,0],[0,ci,0,cj,0,cm],[ci,bi,cj,bj,cm,bm] ])
    D=( E/(1-u**2) )*np.array([ [1,u,0],[u,1,0],[0,0,(1-u)/2] ]) #弹性矩阵
    
    # S=np.dot(D,B)
    # S=np.array([ [bi,u*ci,bj,u*cj,bm,u*cm],[u*bi,ci,u*bj,cj,u*bm,cm],[(1-u)*0.5*ci,(1-u)*0.5*bi,(1-u)*0.5*cj,(1-u)*0.5*bj,(1-u)*0.5*cm,(1-u)*0.5*bm] ])*E/ ( A2*(1-u**2) )
    # k=np.dot(B.T,np.dot(D,B) )*h*A  #方法1：运用矩阵计算

    k=E*h/(4*A*(1-u**2)) *np.array([ 
    [bi*bi+(1-u)*0.5*ci*ci,u*bi*ci+(1-u)*0.5*ci*bi,bi*bj+(1-u)*0.5*ci*cj,u*bi*cj+(1-u)*0.5*ci*bj,bi*bm+(1-u)*0.5*ci*cm,u*bi*cm+(1-u)*0.5*ci*bm],
    [u*ci*bi+(1-u)*0.5*bi*ci,ci*ci+(1-u)*0.5*bi*bi,u*ci*bj+(1-u)*0.5*bi*cj,ci*cj+(1-u)*0.5*bi*bj,u*ci*bm+(1-u)*0.5*bi*cm,ci*cm+(1-u)*0.5*bi*bm],
    
    [bj*bi+(1-u)*0.5*cj*ci,u*bj*ci+(1-u)*0.5*cj*bi,bj*bj+(1-u)*0.5*cj*cj,u*bj*cj+(1-u)*0.5*cj*bj,bj*bm+(1-u)*0.5*cj*cm,u*bj*cm+(1-u)*0.5*cj*bm],
    [u*cj*bi+(1-u)*0.5*bj*ci,cj*ci+(1-u)*0.5*bj*bi,u*cj*bj+(1-u)*0.5*bj*cj,cj*cj+(1-u)*0.5*bj*bj,u*cj*bm+(1-u)*0.5*bj*cm,cj*cm+(1-u)*0.5*bj*bm],

    [bm*bi+(1-u)*0.5*cm*ci,u*bm*ci+(1-u)*0.5*cm*bi,bm*bj+(1-u)*0.5*cm*cj,u*bm*cj+(1-u)*0.5*cm*bj,bm*bm+(1-u)*0.5*cm*cm,u*bm*cm+(1-u)*0.5*cm*bm],
    [u*cm*bi+(1-u)*0.5*bm*ci,cm*ci+(1-u)*0.5*bm*bi,u*cm*bj+(1-u)*0.5*bm*cj,cm*cj+(1-u)*0.5*bm*bj,u*cm*bm+(1-u)*0.5*bm*cm,cm*cm+(1-u)*0.5*bm*bm],
     ]) #方法2：运用坐标计算
    return k
# Element_stiffness_matrix(1,0,0,1,0,0,E=2*10**11,u=0.3,h=1)


# 组装整体刚度矩阵

In [6]:
def assemble_overall_stiffness_matrix(k,i,j,m):
    K_temp = np.zeros((30,30))
    K_temp[2*i-2:2*i,2*i-2:2*i] = k[0:2,0:2]
    K_temp[2*i-2:2*i,2*j-2:2*j] = k[0:2,2:4]
    K_temp[2*i-2:2*i,2*m-2:2*m] = k[0:2,4:6]

    K_temp[2*j-2:2*j,2*i-2:2*i] = k[2:4,0:2]
    K_temp[2*j-2:2*j,2*j-2:2*j] = k[2:4,2:4]
    K_temp[2*j-2:2*j,2*m-2:2*m] = k[2:4,4:6]

    K_temp[2*m-2:2*m,2*i-2:2*i] = k[4:6,0:2]
    K_temp[2*m-2:2*m,2*j-2:2*j] = k[4:6,2:4]
    K_temp[2*m-2:2*m,2*m-2:2*m] = k[4:6,4:6]
    return K_temp


In [7]:
h,b=2,1
K=np.zeros((30,30)) 
k1=Element_stiffness_matrix(h,-b/2,h,0,3*h/4,-b/2,E=2*10**11,u=0.3,h=1)     ;   K+=assemble_overall_stiffness_matrix(k1,1,2,4)
k2=Element_stiffness_matrix(h,0,3*h/4,0,3*h/4,-b/2,E=2*10**11,u=0.3,h=1)    ;   K+=assemble_overall_stiffness_matrix(k2,2,5,4)
k3=Element_stiffness_matrix(h,0,h,b/2,3*h/4,0,E=2*10**11,u=0.3,h=1)         ;   K+=assemble_overall_stiffness_matrix(k3,2,3,5) 
k4=Element_stiffness_matrix(h,b/2,3*h/4,b/2,3*h/4,0,E=2*10**11,u=0.3,h=1)   ;   K+=assemble_overall_stiffness_matrix(k4,3,6,5)
k5=Element_stiffness_matrix(3*h/4,-b/2,3*h/4,0,h/2,-b/2,E=2*10**11,u=0.3,h=1);  K+=assemble_overall_stiffness_matrix(k5,4,5,7)
k6=Element_stiffness_matrix(3*h/4,0,h/2,0,h/2,-b/2,E=2*10**11,u=0.3,h=1)    ;   K+=assemble_overall_stiffness_matrix(k6,5,8,7)
k7=Element_stiffness_matrix(3*h/4,0,3*h/4,b/2,h/2,0,E=2*10**11,u=0.3,h=1)   ;   K+=assemble_overall_stiffness_matrix(k7,5,6,8)
k8=Element_stiffness_matrix(3*h/4,b/2,h/2,b/2,h/2,0,E=2*10**11,u=0.3,h=1)   ;   K+=assemble_overall_stiffness_matrix(k8,6,9,8)
k9=Element_stiffness_matrix(h/2,-b/2,h/2,0,h/4,-b/2,E=2*10**11,u=0.3,h=1)   ;   K+=assemble_overall_stiffness_matrix(k9,7,8,10)
k10=Element_stiffness_matrix(h/2,0,h/4,0,h/4,-b/2,E=2*10**11,u=0.3,h=1)     ;   K+=assemble_overall_stiffness_matrix(k10,8,11,10)
k11=Element_stiffness_matrix(h/2,0,h/2,b/2,h/4,0,E=2*10**11,u=0.3,h=1)      ;   K+=assemble_overall_stiffness_matrix(k11,8,9,11)
k12=Element_stiffness_matrix(h/2,b/2,h/4,b/2,h/4,0,E=2*10**11,u=0.3,h=1)    ;   K+=assemble_overall_stiffness_matrix(k12,9,12,11)
k13=Element_stiffness_matrix(h/4,-b/2,h/4,0,0,-b/2,E=2*10**11,u=0.3,h=1)    ;   K+=assemble_overall_stiffness_matrix(k13,10,11,13)
k14=Element_stiffness_matrix(h/4,0,0,0,0,-b/2,E=2*10**11,u=0.3,h=1)         ;   K+=assemble_overall_stiffness_matrix(k14,11,14,13)
k15=Element_stiffness_matrix(h/4,0,h/4,b/2,0,0,E=2*10**11,u=0.3,h=1)        ;   K+=assemble_overall_stiffness_matrix(k15,11,12,14)
k16=Element_stiffness_matrix(h/4,b/2,0,b/2,0,0,E=2*10**11,u=0.3,h=1)        ;   K+=assemble_overall_stiffness_matrix(k16,12,15,14)
#K.shape

# 用整体载荷矩阵 边界条件 求解节点位移矩阵

In [8]:
#整体力矩阵
F=np.array([-25,0, 0,0, 25,0, -50,0, 0,0, 50,0, -50,0, 0,0, 50,0, -50,0, 0,0, 50,0, 150,200, 0,0, 50,0 ])

In [38]:
#位移边界条件  1 2 3 节点的位移为0 因此需要划去
K_end=K[6:,6:] #只取需要的部分
# K_end.shape
F_end=F[6:]

d=np.dot( np.linalg.inv(K_end) , F_end )
print("4到15节点的节点位移矩阵为\n",d) #从4节点开始的节点位移矩阵
# d

4到15节点的节点位移矩阵为
 [-3.04238827e-09  3.03149721e-09  4.17273385e-10  2.88953133e-09
  4.21134408e-09  3.64317248e-09 -4.98436806e-09  9.02699849e-09
  1.04050090e-09  9.12522927e-09  7.12369228e-09  9.73806801e-09
 -5.64591103e-09  1.69935465e-08  1.62362489e-09  1.71719930e-08
  8.77690479e-09  1.74088687e-08 -4.80707774e-09  2.64038876e-08
  1.72960503e-09  2.53649634e-08  9.37469391e-09  2.54171743e-08]


In [39]:
print("4-15结点x向位移\n",d[0::2] ) #4-15结点x向位移

4-15结点x向位移
 [-3.04238827e-09  4.17273385e-10  4.21134408e-09 -4.98436806e-09
  1.04050090e-09  7.12369228e-09 -5.64591103e-09  1.62362489e-09
  8.77690479e-09 -4.80707774e-09  1.72960503e-09  9.37469391e-09]


In [41]:
print("4-15结点y向位移\n",d[1::2] ) #4-15节点y向位移

4-15结点y向位移
 [3.03149721e-09 2.88953133e-09 3.64317248e-09 9.02699849e-09
 9.12522927e-09 9.73806801e-09 1.69935465e-08 1.71719930e-08
 1.74088687e-08 2.64038876e-08 2.53649634e-08 2.54171743e-08]


# 获取每个的单元位移矩阵函数  


In [12]:
def node_dis_select(d,i):       # 获取某节点位移  输入：总的位移矩阵 
    if i<4:
        result=np.array([0,0])
    else:
        result=d[(2*i-8):(2*i-6)]

    
    return result
# type( node_dis_select(d,6) )
def element_dis(i,j,m):             #获取单元位移矩阵  输入： 节点编号 i j m   调用：获取节点位移函数
    temp_matrx=np.append( node_dis_select(d,i), node_dis_select(d,j) )
    result=np.append( temp_matrx, node_dis_select(d,m)  )
    return result
# element_dis(0,1,4)

#### 单元应变矩阵 应力矩阵 实验以确定规律编写函数文件

In [13]:
#1单元 位移矩阵 应力矩阵
# d1=np.array([0,0,0,0,-3.04238827e-09,  3.03149721e-09])
d1=element_dis(0,1,4)
B1=Geometric_matrix(h,-b/2,h,0,3*h/4,-b/2)
yingbian1=np.dot(B1,d1)
print("1单元应变为",yingbian1)  

S1=Stress_matrix(h,-b/2,h,0,3*h/4,-b/2,E=2*10**11,u=0.3)
thegma1=np.dot(S1,d1)
print("1单元应力为",thegma1)        #[thegma_x  thegma_y tao_xy]

1单元应变为 [ 1.52119414e-09  0.00000000e+00 -1.51574861e-09]
1单元应力为 [1337.31352649  401.19405795 -466.38418646]


In [14]:
#5单元 位移矩阵 应力矩阵
d5=element_dis(4,5,7)   # i j m
B5=Geometric_matrix(3*h/4,-b/2,3*h/4,0,h/2,-b/2)
yingbian5=np.dot(B5,d5)
print("5单元应变为",yingbian5)  

S5=Stress_matrix(3*h/4,-b/2,3*h/4,0,h/2,-b/2,E=2*10**11,u=0.3)
thegma5=np.dot(S5,d5)
print("5单元应力为",thegma5)        #[thegma_x  thegma_y tao_xy]

5单元应变为 [ 9.70989892e-10 -7.09829407e-11 -1.26791981e-09]
5单元应力为 [ 834.89671213  193.68266105 -390.12917248]


# 单元应变矩阵 和 单元应力矩阵 获取函数

In [15]:
def stress_matrx_elem(n,i,j,m,xi,yi,xj,yj,xm,ym,E=2*10**11,u=0.3):
    dn=element_dis(i,j,m)
    Bn=Geometric_matrix(xi,yi,xj,yj,xm,ym)
    yingbian=np.dot(Bn,dn)
    print(n,"单元应变为:",yingbian) 

    Sn=Stress_matrix(xi,yi,xj,yj,xm,ym,E=2*10**11,u=0.3)
    thegma=np.dot(Sn,dn)
    print(n,"单元应力为：",thegma)

    return

# 各单元的 单元应变矩阵 和 单元应力矩阵

In [16]:
stress_matrx_elem(1,1,2,4,h,-b/2,h,0,3*h/4,-b/2,E=2*10**11,u=0.3)

1 单元应变为: [ 1.52119414e-09  0.00000000e+00 -1.51574861e-09]
1 单元应力为： [1337.31352649  401.19405795 -466.38418646]


In [17]:
stress_matrx_elem(2,2,5,4,h,0,3*h/4,0,3*h/4,-b/2,E=2*10**11,u=0.3)

2 单元应变为: [-2.08636693e-10 -7.09829407e-11  2.85065164e-10]
2 单元应力为： [-202.13764832 -117.42764708   87.71235812]


In [18]:
stress_matrx_elem(3,2,3,5,h,0,h,b/2,3*h/4,0,E=2*10**11,u=0.3)

3 单元应变为: [-2.08636693e-10  0.00000000e+00 -1.44476567e-09]
3 单元应力为： [-183.41687274  -55.02506182 -444.54328162]


In [19]:
stress_matrx_elem(4,3,6,5,h,b/2,3*h/4,b/2,3*h/4,0,E=2*10**11,u=0.3)

4 单元应变为: [-2.10567204e-09  3.76820576e-10  7.54491074e-11]
4 单元应力为： [-1751.75900543  -224.07124069    23.21510996]


In [20]:
stress_matrx_elem(5,4,5,7,3*h/4,-b/2,3*h/4,0,h/2,-b/2,E=2*10**11,u=0.3)

5 单元应变为: [ 9.70989892e-10 -7.09829407e-11 -1.26791981e-09]
5 单元应力为： [ 834.89671213  193.68266105 -390.12917248]


In [21]:
stress_matrx_elem(6,5,8,7,3*h/4,0,h/2,0,h/2,-b/2,E=2*10**11,u=0.3)

6 单元应变为: [-3.11613756e-10  4.91153908e-11 -1.05414494e-10]
6 单元应力为： [-260.99264934  -39.00548214  -32.43522892]


In [22]:
stress_matrx_elem(7,5,6,8,3*h/4,0,3*h/4,b/2,h/2,0,E=2*10**11,u=0.3)

7 单元应变为: [-3.11613756e-10  3.76820576e-10 -1.22081362e-09]
7 单元应力为： [-174.56490815  249.08698849 -375.63496076]


In [23]:
stress_matrx_elem(8,6,9,8,3*h/4,b/2,h/2,b/2,h/2,0,E=2*10**11,u=0.3)

8 单元应变为: [-1.45617410e-09  3.06419370e-10 -5.85207299e-12]
8 单元应力为： [-1199.33915464  -114.66625012    -1.80063784]


In [24]:
stress_matrx_elem(9,7,8,10,h/2,-b/2,h/2,0,h/4,-b/2,E=2*10**11,u=0.3)

9 单元应变为: [ 3.30771487e-10  4.91153908e-11 -9.70839522e-10]
9 单元应力为： [ 303.74163035  130.41480176 -298.71985286]


In [25]:
stress_matrx_elem(10,8,11,10,h/2,0,h/4,0,h/4,-b/2,E=2*10**11,u=0.3)

10 单元应变为: [-2.91561997e-10  8.92232614e-11 -3.88613908e-10]
10 单元应力为： [-232.78682918    1.54256036 -119.5735103 ]


In [26]:
stress_matrx_elem(11,8,9,11,h/2,0,h/2,b/2,h/4,0,E=2*10**11,u=0.3)

11 单元应变为: [-2.91561997e-10  3.06419370e-10 -9.81786177e-10]
11 单元应力为： [-175.50433891  192.4841946  -302.08805452]


In [27]:
stress_matrx_elem(12,9,12,11,h/2,b/2,h/4,b/2,h/4,0,E=2*10**11,u=0.3)

12 单元应变为: [-8.26606254e-10  1.18437843e-10 -2.58760393e-10]
12 单元应力为： [-695.45046227 -113.88486453  -79.61858232]


In [28]:
stress_matrx_elem(13,10,11,13,h/4,-b/2,h/4,0,0,-b/2,E=2*10**11,u=0.3)

13 单元应变为: [-4.19416644e-10  8.92232614e-11 -1.07040258e-09]
13 单元应力为： [-345.18651905  -32.1773466  -329.3546391 ]


In [29]:
stress_matrx_elem(14,11,14,13,h/4,0,0,0,0,-b/2,E=2*10**11,u=0.3)

14 单元应变为: [-5.29900691e-11 -5.19462077e-10 -8.28143813e-10]
14 单元应力为： [-183.58566358 -470.6453609  -254.81348095]


In [30]:
stress_matrx_elem(15,11,12,14,h/4,0,h/4,b/2,0,0,E=2*10**11,u=0.3)

15 单元应变为: [-5.29900691e-11  1.18437843e-10 -5.19845250e-10]
15 单元应力为： [ -15.34832206   90.14577753 -159.95238464]


In [31]:
stress_matrx_elem(16,12,15,14,h/4,b/2,0,b/2,0,0,E=2*10**11,u=0.3)

16 单元应变为: [-2.98894558e-10  2.61054416e-11 -1.81608360e-10]
16 单元应力为： [-255.87949531  -55.87949531  -55.87949531]


In [32]:
def maxtha(x,y):
    thegmax=1200*y-3600*x*y-200
    txy=1800*y**2-350
    maxtha=thegmax/2 + ( (thegmax/2)**2 + txy**2)**0.5
    return maxtha
maxtha(0,0)

264.0054944640259